In [17]:

from openpyxl import load_workbook
workbook = load_workbook(filename='../etc/M1M Modbus map V1.2.xlsx')

from string import Template
ai_template = Template(
    '''
    record(ai, "${pv_name}") {
        field(DESC, "")
        field(ASLO, "${resolution}")
        field(DTYP,"asynInt32")
        field(INP,"@asyn($(PORT), ${register_addr})${data_type}")
        field(SCAN,".1 second")
    }
    '''
)

sheet = workbook['M1M registers map']
sheet.cell(row=10, column=6).value

ColNames = {}
Current  = 0
for COL in sheet.iter_cols(1, sheet.max_column):
    ColNames[COL[0].value] = Current
    Current += 1

In [19]:
# Write db file

def to_camel(string: str):
    if type(string) != None:
        output = ''
        for _ in range(len(string)):
            if _>0 and string[_-1]==' ':
                output += string[_].upper()
            else:
                output += string[_]
        output = output.replace(" ", "")
        return output

with open('../ioc/database/st.db', 'w') as file:

    for row_cells in sheet.iter_rows(2, sheet.max_row):
        
        if row_cells[ColNames['Monitored']].value == 'yes':
            
            if row_cells[ColNames['Quantity/Functionality']].value != None:
                
                if row_cells[ColNames['Data Type']].value == 'Unsigned' and row_cells[ColNames['Length']].value == 2:
                    
                    print(
                    row_cells[ColNames['Quantity/Functionality']].value, row_cells[ColNames['Quantity/Functionality']],
                    'Unsigned 2'
                    )
                
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = '',
                        resolution = row_cells[ColNames['Length']].value)
                            )
                
                elif row_cells[ColNames['Data Type']].value == 'Unsigned' and row_cells[ColNames['Length']].value == 1:
                    
                    print(
                    row_cells[ColNames['Quantity/Functionality']].value, row_cells[ColNames['Quantity/Functionality']],
                    'Unsigned 2'
                    )
                
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT16',
                        resolution = row_cells[ColNames['Length']].value)
                            )
                
                elif row_cells[ColNames['Data Type']].value == 'Signed' and row_cells[ColNames['Length']].value == 2:
                    
                    print(
                    row_cells[ColNames['Quantity/Functionality']].value, row_cells[ColNames['Quantity/Functionality']],
                    'Unsigned 2'
                    )
                    
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT32_BE',
                        resolution = row_cells[ColNames['Length']].value)
                            )
                
                elif row_cells[ColNames['Data Type']].value == 'Signed' and row_cells[ColNames['Length']].value == 1:
                    
                    print(
                    row_cells[ColNames['Quantity/Functionality']].value, row_cells[ColNames['Quantity/Functionality']],
                    'Unsigned 2'
                    )
                    
                    file.write(ai_template.safe_substitute(
                        pv_name = to_camel(row_cells[ColNames['Quantity/Functionality']].value),
                        register_addr = row_cells[ColNames['Register\n(Dec)']].value,
                        data_type = 'INT32_BE',
                        resolution = row_cells[ColNames['Length']].value)
                            )


Three phase system  <Cell 'M1M registers map'.A7> Unsigned 2
Phase Voltage L1 <Cell 'M1M registers map'.A8> Unsigned 2
Phase Voltage L2 <Cell 'M1M registers map'.A9> Unsigned 2
Phase Voltage L3 <Cell 'M1M registers map'.A10> Unsigned 2
Line voltage L1-L2 <Cell 'M1M registers map'.A11> Unsigned 2
Line voltage L3-L2 <Cell 'M1M registers map'.A12> Unsigned 2
Line voltage L1-L3 <Cell 'M1M registers map'.A13> Unsigned 2
Three phase Current <Cell 'M1M registers map'.A14> Unsigned 2
Current L1 <Cell 'M1M registers map'.A15> Unsigned 2
Current L2 <Cell 'M1M registers map'.A16> Unsigned 2
Current L3 <Cell 'M1M registers map'.A17> Unsigned 2
Current N <Cell 'M1M registers map'.A18> Unsigned 2
Active Power total <Cell 'M1M registers map'.A19> Unsigned 2
Reactive Power total <Cell 'M1M registers map'.A23> Unsigned 2
Apparent Power total <Cell 'M1M registers map'.A27> Unsigned 2
Frequency <Cell 'M1M registers map'.A31> Unsigned 2
Voltage Angle L1 <Cell 'M1M registers map'.A36> Unsigned 2
Voltage An

In [15]:
for row_cells in sheet.iter_rows(2, sheet.max_row):
    print(row_cells[ColNames['Quantity/Functionality']].value)

Active energy +
Active energy -
Reactive energy +
Reactive energy -
Apparent energy
Three phase system 
Phase Voltage L1
Phase Voltage L2
Phase Voltage L3
Line voltage L1-L2
Line voltage L3-L2
Line voltage L1-L3
Three phase Current
Current L1
Current L2
Current L3
Current N
Active Power total
Active Power L1
Active Power L2
Active power L3 
Reactive Power total
Reactive Power L1 
Reactive Power L2 
Reactive Power L3
Apparent Power total
Apparent Power L1
Apparent Power L2
Apaprent power L3 
Frequency
Power Total Angle
Power Angle L1
Power Angle L2
Power Angle L3
Voltage Angle L1
Voltage Angle L2
Voltage Angle L3
Current Angle L1
Current Angle L2
Current Angle L3
Power factor total
Power factor L1
Power factor L2
Power factor L3
Displacement factor total
Displacement factor L1
Displacement factor L2
Displacement factor L3
Average current L1
Average current L2
Average current L3
Average current neutral
Average voltage L1
Average voltage L2
Average voltage L3
Average voltage L1-L2
Average

In [ ]:
# Testing serial

import serial, time

ser = serial.Serial('/dev/pts/5', baudrate=9600, bytesize=serial.EIGHTBITS, parity=serial.PARITY_NONE)
ser.write(b'\x02\x03\x5B\x02\x00\x02\x76\xDC')

size = 0
answer = ''
# while size < 9:
#     print(ser.read())
#     size +=1
# print(answer)

time.sleep(1)
print(ser.readline())

ser.close()

In [ ]:
from pymodbus.client import ModbusSerialClient

client = ModbusSerialClient(
    method='rtu',
    port='/dev/pts/14',
    baudrate=115200,
    timeout=3,
    parity='N',
    stopbits=1,
    bytesize=8
)

if client.connect():  # Trying for connect to Modbus Server/Slave
    '''Reading from a holding register with the below content.'''
    res = client.read_holding_registers(address=23298, count=2, slave=2)

    '''Reading from a discrete register with the below content.'''
    # res = client.read_discrete_inputs(address=1, count=1, unit=1)

    if not res.isError():
        print(res.registers)
    else:
        print(res)

else:
    print('Cannot connect to the Modbus Server/Slave')